In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# Regularización

### Nota:

En este ejercicio vamos a escalar las features del dataset usando `MinMaxScaler` con el objetivo de que tengan un ejercicio resuelto de ejemplo con una alternativa a `StandardScaler`, no porque consideremos que en este problema `MinMaxScaler` resulte en una mejor performance que `StandardScaler`.

---

Aunque la normalización a través de min-max es una técnica de uso común que es útil cuando necesitamos valores en un intervalo acotado, la estandarización puede ser más práctica para muchos algoritmos de aprendizaje automático. 

La razón es que muchos modelos lineales inicializan las ponderaciones en O o valores aleatorios pequeños cercanos a 0.

Usando la estandarización centramos las columnas de features en la media 0 con el desvío estándar 1, así las columnas de features adoptan la forma de una distribución normal, lo que facilita el aprendizaje de los pesos. 

Además, la estandarización mantiene información útil sobre los valores atípicos y hace que el algoritmo sea menos sensible a ellos en contraste con el escalado min-max, que escala los datos a un rango limitado de valores.


## Imports

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn import metrics

import statsmodels.api as sm
from statsmodels.tools import eval_measures


## Dataset

Este dataset contiene los precios y otros atributos de casi 54.000 diamantes.

Sus features son:

* **price**: price in US dollars (\$326--\$18,823).  **Esta es la variable target**.

* carat: weight of the diamond (0.2--5.01)

* cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)

* color: diamond colour, from J (worst) to D (best)

* clarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

* x: length in mm (0--10.74)

* y: width in mm (0--58.9)

* z: depth in mm (0--31.8)

* depth: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)

* table: width of top of diamond relative to widest point (43--95)

Fuente: https://www.kaggle.com/shivam2503/diamonds

## Leemos los datos

In [ ]:
data = pd.read_csv('../Data/diamonds.csv')
data.head()

In [ ]:
data.shape

## Ejercicio 1

Normalicemos las features y creemos las variables dummies necesarias para poder entrenar un modelo de regresión para predecir el valor de `price` para cada registro

https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

In [ ]:
categoricals = ['cut', 'color', 'clarity']

enc = OneHotEncoder(drop='first')
X = data[categoricals]
enc.fit(X)
enc.categories_

In [ ]:
dummies = enc.transform(X).toarray()
dummies

In [ ]:
dummies.shape

In [ ]:
dummies_df = pd.DataFrame(dummies)
dummies_df

In [ ]:
col_names = [categoricals[i] + '_' + enc.categories_[i] for i in range(len(categoricals)) ]

col_names

In [ ]:
col_names_drop_first = [sublist[i] for sublist in col_names for i in range(len(sublist)) if i != 0]
col_names_drop_first

In [ ]:
dummies_df.columns = col_names_drop_first
dummies_df

Otra opción es usar `get_dummies`

https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [ ]:
pd.get_dummies(X, drop_first = True)

Ahora estandarizamos las features numéricas:

`carat` `depth` `table` `x` `y` `z`

usando  `MinMaxScaler`

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler


In [ ]:
numericals = ['carat', 'depth', 'table', 'x', 'y', 'z']

X = data[numericals]

scaler = MinMaxScaler()
scaler.fit(X)

std_numerical_data = scaler.transform(X)
std_df = pd.DataFrame(std_numerical_data)
std_df.columns = [i + '_std' for i in numericals]
std_df

Entonces nuestro dataset de features serán las variables dummies y las variables numéricas estandarizadas

In [ ]:
X = pd.concat([dummies_df, std_df], axis = 1)
X

Y la variable target es `price`

In [ ]:
y = data.price

## Ejercicio 2

Separemos el conjunto en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 117)

## Ejercicio 3

Ajustemos una regresión lineal múltiple con los datos del conjunto de entrenamiento usando statsmodels y evaluemos la significancia de cada uno de los coeficientes

In [ ]:
# Tenemos que agregar explícitamente a una constante:
X_train_sm = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train_sm).fit()

model.summary()

In [ ]:
no_reg_model_params = model.params

Vemos que los p-value de los coeficientes de y_std y z_std son altos, por lo tanto no podemos rechazar la hipótesis nula que dice que los coeficientes de esas dos variables son 0.

In [ ]:
sm_prediction_train = model.predict(X_train_sm)
print(eval_measures.rmse(y_train, sm_prediction_train))

X_test_sm = sm.add_constant(X_test)
sm_prediction_test = model.predict(X_test_sm)
print(eval_measures.rmse(y_test, sm_prediction_test))


## Ejercicio 4

Ajustamos el modelo aplicando regularización de Lasso y validación cruzada para estimar el mejor valor de $\alpha$ para este problema

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

¿Cuál es el mejor valor de $\alpha$ para este problema?

¿Cuál es el score obtenido ($R^2$) para este modelo en entrenamiento?

In [ ]:
# Definimos el rango de de búsqueda del hiperparametro explicitamente
lm_lasso = linear_model.LassoCV(alphas=[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 5, 10],\
                                        normalize = False, cv = 5) 

model_cv = lm_lasso.fit(X_train, y_train)

model_cv.score(X_train, y_train)

In [ ]:
model_cv.coef_

In [ ]:
model_cv.intercept_

In [ ]:
model_cv.alpha_

In [ ]:
model_cv.score(X_train, y_train)

## Ejercicio 5 

Ajustemos los datos de entrenamiento con una regresión con regularización de Lasso para el valor de $\alpha$ calculado en el punto anterior usando statsmodels.

Usemos scatterplots para mostrar 

* los valores de los coeficientes de la regresión lineal múltiple obtenidos en el Ejercicio 3, y los valores de los coeficientes de la regresión lineal con regularización de Lasso para el modelo entrenado.

* los valores de los residuos en entrenamiento resultado del Ejercicio 3, y los residuos en entrenamiento para el modelo con regularización.

https://www.statsmodels.org/0.6.1/generated/statsmodels.regression.linear_model.OLS.fit_regularized.html

In [ ]:
best_alpha = model_cv.alpha_

#L1_wt : 0, the fit is ridge regression. 1, the fit is the lasso 

no_reg_model = sm.OLS(y_train, X_train_sm)

reg_model = no_reg_model.fit_regularized(alpha = best_alpha, L1_wt = 1)


In [ ]:
 reg_model.params

In [ ]:
sns.scatterplot(x=reg_model.params, y=no_reg_model_params);

In [ ]:
reg_residuals = y_train - reg_model.fittedvalues

linear_residuals = y_train - model.fittedvalues

sns.scatterplot(x = reg_residuals, y = linear_residuals)

## Ejercicio 6

Usandos statsmodels y scikit-learn calculemos la performance en test del modelo construído y comparemos los resultados de las dos bibliotecas usando como métricas el error absoluto medio (MAE) y la raiz del error cuadrático medio (RMSE) 

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:

sm_prediction = reg_model.predict(X_test_sm)
sm_prediction

In [ ]:
skl_lasso = linear_model.Lasso(alpha = best_alpha, fit_intercept=True, normalize=False)

skl_lasso = skl_lasso.fit(X= X_train, y = y_train)

skl_prediction = skl_lasso.predict(X_test)


In [ ]:
skl_residuals = y_test - skl_prediction

sm_residuals = y_test - sm_prediction

sns.scatterplot(x = skl_residuals, y = sm_residuals)

In [ ]:

lasso_coef = np.insert(skl_lasso.coef_, 0, skl_lasso.intercept_)

sns.scatterplot(x = lasso_coef, y = reg_model.params);


In [ ]:
lasso_coef

In [ ]:
reg_model.params

Métricas en `statsmodels`

https://www.statsmodels.org/stable/generated/statsmodels.tools.eval_measures.rmse.html

https://www.statsmodels.org/stable/generated/statsmodels.tools.eval_measures.meanabs.html

In [ ]:
eval_measures.rmse(y_test, sm_prediction)

In [ ]:
eval_measures.meanabs(y_test, sm_prediction)

In [ ]:
# de scikit-learn
metrics.r2_score(y_test, sm_prediction)

Métricas en `scikit-learn`

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

In [ ]:
np.sqrt(metrics.mean_squared_error(y_test, skl_prediction))

In [ ]:
metrics.mean_absolute_error(y_test, skl_prediction)

In [ ]:
# de scikit-learn
metrics.r2_score(y_test, skl_prediction)

## Referencias

https://www.kaggle.com/yogendran/intro-to-linear-ridge-and-lasso-regressions
    
https://towardsdatascience.com/intro-to-regularization-with-ridge-and-lasso-regression-with-sklearn-edcf4c117b7a